# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import tensorflow as tf
import numpy as np


%matplotlib inline

USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models.
print_every = 100
print('Using device: ', device)

Using device:  /device:GPU:0


# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170498071/170498071 [==============================] - 4s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [5]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    # with tf.device(device):
    scores = model(x)
    print(scores.shape)

test_TwoLayerFC()

(64, 10)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [6]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # Определение слоев для сверточной нейронной сети.
        ########################################################################
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, x, training=False):
        ########################################################################
        # Прямой проход для сверточной нейронной сети.
        ########################################################################
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [7]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [8]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [9]:

hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1488828659057617, Accuracy: 10.9375, Val Loss: 2.684116840362549, Val Accuracy: 14.5
Iteration 100, Epoch 1, Loss: 2.2259490489959717, Accuracy: 29.022275924682617, Val Loss: 1.872597098350525, Val Accuracy: 39.20000076293945
Iteration 200, Epoch 1, Loss: 2.0744333267211914, Accuracy: 32.26057434082031, Val Loss: 1.8415042161941528, Val Accuracy: 39.29999923706055
Iteration 300, Epoch 1, Loss: 2.003807544708252, Accuracy: 33.892234802246094, Val Loss: 1.8820589780807495, Val Accuracy: 37.29999923706055
Iteration 400, Epoch 1, Loss: 1.9349920749664307, Accuracy: 35.750465393066406, Val Loss: 1.7254904508590698, Val Accuracy: 41.900001525878906
Iteration 500, Epoch 1, Loss: 1.8890869617462158, Accuracy: 36.829463958740234, Val Loss: 1.6493014097213745, Val Accuracy: 44.60000228881836
Iteration 600, Epoch 1, Loss: 1.860552430152893, Accuracy: 37.65599060058594, Val Loss: 1.6781830787658691, Val Accuracy: 43.0
Iteration 700, Epoch 1, Loss: 1.8342262506484985, 

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [10]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3979787826538086, Accuracy: 4.6875, Val Loss: 2.4270572662353516, Val Accuracy: 8.399999618530273
Iteration 100, Epoch 1, Loss: 1.8665436506271362, Accuracy: 33.66336441040039, Val Loss: 1.6504309177398682, Val Accuracy: 43.29999923706055
Iteration 200, Epoch 1, Loss: 1.7202601432800293, Accuracy: 39.1091423034668, Val Loss: 1.471838116645813, Val Accuracy: 49.70000076293945
Iteration 300, Epoch 1, Loss: 1.6374460458755493, Accuracy: 41.60610580444336, Val Loss: 1.4420742988586426, Val Accuracy: 48.89999771118164
Iteration 400, Epoch 1, Loss: 1.5725393295288086, Accuracy: 43.905860900878906, Val Loss: 1.3775074481964111, Val Accuracy: 50.70000457763672
Iteration 500, Epoch 1, Loss: 1.5261958837509155, Accuracy: 45.44348907470703, Val Loss: 1.339013934135437, Val Accuracy: 51.70000076293945
Iteration 600, Epoch 1, Loss: 1.496576189994812, Accuracy: 46.49282455444336, Val Loss: 1.2882561683654785, Val Accuracy: 55.19999694824219
Iteration 700, Epoch 1, Loss:

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [11]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.3933353424072266, Accuracy: 4.6875, Val Loss: 3.020481586456299, Val Accuracy: 13.0
Iteration 100, Epoch 1, Loss: 2.225173234939575, Accuracy: 29.2852725982666, Val Loss: 1.9103121757507324, Val Accuracy: 37.099998474121094
Iteration 200, Epoch 1, Loss: 2.07135009765625, Accuracy: 32.6881217956543, Val Loss: 1.820311188697815, Val Accuracy: 39.79999923706055
Iteration 300, Epoch 1, Loss: 1.9982608556747437, Accuracy: 34.535919189453125, Val Loss: 1.864005208015442, Val Accuracy: 38.10000228881836
Iteration 400, Epoch 1, Loss: 1.926121473312378, Accuracy: 36.36611557006836, Val Loss: 1.7324796915054321, Val Accuracy: 42.20000076293945
Iteration 500, Epoch 1, Loss: 1.8825420141220093, Accuracy: 37.356536865234375, Val Loss: 1.6714897155761719, Val Accuracy: 43.5
Iteration 600, Epoch 1, Loss: 1.8535958528518677, Accuracy: 38.18375778198242, Val Loss: 1.6830545663833618, Val Accuracy: 41.400001525878906
Iteration 700, Epoch 1, Loss: 1.8286441564559937, Accurac

Альтернативный менее гибкий способ обучения:

In [12]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.7066 - sparse_categorical_accuracy: 0.4267


[1.7066301107406616, 0.42669999599456787]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [13]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3882036209106445, Accuracy: 6.25, Val Loss: 2.31231427192688, Val Accuracy: 11.90000057220459
Iteration 100, Epoch 1, Loss: 2.2553277015686035, Accuracy: 17.295793533325195, Val Loss: 2.2179737091064453, Val Accuracy: 19.100000381469727
Iteration 200, Epoch 1, Loss: 2.221480369567871, Accuracy: 19.348569869995117, Val Loss: 2.155139923095703, Val Accuracy: 23.899999618530273
Iteration 300, Epoch 1, Loss: 2.1951794624328613, Accuracy: 20.468231201171875, Val Loss: 2.1099038124084473, Val Accuracy: 25.700000762939453
Iteration 400, Epoch 1, Loss: 2.164639472961426, Accuracy: 21.9217586517334, Val Loss: 2.065847635269165, Val Accuracy: 28.200000762939453
Iteration 500, Epoch 1, Loss: 2.142360210418701, Accuracy: 22.879241943359375, Val Loss: 2.037090301513672, Val Accuracy: 28.200000762939453
Iteration 600, Epoch 1, Loss: 2.1209280490875244, Accuracy: 23.765079498291016, Val Loss: 2.009662628173828, Val Accuracy: 29.80000114440918
Iteration 700, Epoch 1, Loss

In [14]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.4156 - sparse_categorical_accuracy: 0.4935


[1.4155585765838623, 0.4934999942779541]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [15]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [16]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.995513677597046, Accuracy: 15.625, Val Loss: 2.8921446800231934, Val Accuracy: 12.700000762939453
Iteration 100, Epoch 1, Loss: 2.2282817363739014, Accuracy: 28.326114654541016, Val Loss: 1.9204689264297485, Val Accuracy: 36.89999771118164
Iteration 200, Epoch 1, Loss: 2.068678140640259, Accuracy: 32.1750602722168, Val Loss: 1.8840391635894775, Val Accuracy: 37.0
Iteration 300, Epoch 1, Loss: 1.9973726272583008, Accuracy: 34.26079559326172, Val Loss: 1.8862063884735107, Val Accuracy: 36.599998474121094
Iteration 400, Epoch 1, Loss: 1.926774501800537, Accuracy: 36.2258415222168, Val Loss: 1.746034026145935, Val Accuracy: 40.400001525878906
Iteration 500, Epoch 1, Loss: 1.8849895000457764, Accuracy: 37.20683670043945, Val Loss: 1.6628777980804443, Val Accuracy: 42.39999771118164
Iteration 600, Epoch 1, Loss: 1.8547316789627075, Accuracy: 38.06936264038086, Val Loss: 1.6916673183441162, Val Accuracy: 41.5
Iteration 700, Epoch 1, Loss: 1.8302843570709229, Accu

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [17]:
class _IdentityBlock(tf.keras.Model):
    """Identity block utilizing skip connections."""

    def __init__(self, out_channels):
        super().__init__()
        """Initializes the identiy block.

        Here we simply initialize 2 layers which process the input and after the output
        is produces it is added together with the input whcih is the final output.

        Args:
            out_channels (int): The number of activation maps this block should produce
        """
        # Acts as Kaiming weight initalization
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Part 1 of the convolution, normalization and non-linearity
        self.conv1 = tf.keras.layers.Conv2D(out_channels, 3, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm1 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu1 = tf.keras.layers.Activation('relu')

        # Part 2 of the convolution, normalization and non-linearity
        self.conv2 = tf.keras.layers.Conv2D(out_channels, 3, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm2 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu2 = tf.keras.layers.Activation('relu')

        # Add layer will add together the input and the output
        self.add = tf.keras.layers.Add()

    def call(self, x, training=False):
        """Performs forward pass on the given input.

        Args:
            x (Tensor):      The input of dimensions (N, H, W, C)
            training (bool): Indicates whether the forward pass happens in the training mode

        Returns:
              out (Tensor): Output data of dim (N, H, W, C)
        """
        x_skip = tf.identity(x)                   # prepare to add the input to the output
        x = self.relu1(self.norm1(self.conv1(x))) # pass input through the first layer
        x = self.norm2(self.conv2(x))             # pass input through the second layer (without ReLU)
        out = self.relu2(self.add([x, x_skip]))   # perform ReLU on the processed input added with the raw input

        return out


class ResNet(tf.keras.Model):

    def __init__(self, in_channels=32, block_config=(2, 2, 2, 2), num_classes=10):
        """Initializes the residual network.

        The first layer produces `in_channels` activation maps which are then fed to a
        sequence of blocks containing a specified number of identity sub-blocks (first
        block is always *_BottleneckBlock*). At the end the _global average pooling_
        layer is used to flatten the activations for the linear softmax classifier.

        Args:
            in_channels (int):    The number of channels to extract after the first convolution
            block_config (tuple): The number of layers each bloack should have in sequence
            num_classes (int):    The total number of classes
        """
        super().__init__()

        # Acts as Kaiming weight initalization
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Prepare the input for the chains of identity blocks
        self.features = tf.keras.Sequential([
            tf.keras.layers.Conv2D(in_channels, 5, padding='same', use_bias=False, kernel_initializer=initializer),
            tf.keras.layers.BatchNormalization(axis=3),
            tf.keras.layers.Activation('relu'),
        ])

        num_features = in_channels # num feaure maps to produce after each group of identity blocks

        # Loop through every group of blocks
        for i, num_layers in enumerate(block_config):
            # Use bottleneck block as the first block in every group (except first)
            if i != 0:
                self.features.add(_BottleneckBlock(num_features))
            else:
                self.features.add(_IdentityBlock(num_features))

            # Create the specified number of identity blocks for i'th group
            for j in range(num_layers-1):
                self.features.add(_IdentityBlock(num_features))

            num_features *= 2 # increase the nuber of features to be produced

        # Flatten the final activation maps using global average pooling
        self.features.add(tf.keras.layers.GlobalAveragePooling2D())

        # Softmax classifier is used as the final layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)

    def call(self, x, training=False):
        """Performs forward pass on the given input.

        Args:
            x (Tensor):      The input of dimensions (N, H, W, C)
            training (bool): Indicates whether the forward pass happens in the training mode

        Returns:
            out (Tensor): Output data of dim (N, 10)
        """
        out = self.features(x)     # Get the extracted features for the linear classfier
        out = self.classifier(out) # Perform classification with softmax activation

        return out
class _BottleneckBlock(tf.keras.Model):
    """Same as identity block except it reduces the spacial area before processing the input."""

    def __init__(self, out_channels):
        """Initializes the bottleneck block.

        Unlike *_IdentityBlock*, the first convolution here reduces the spacial size of the input
        by a factor of `2`. Then, it performs the main convolution after which the output maps
        are added together with the input maps to produce final activations.

        Args:
            out_channels (int): The number of activation maps this block should produce
        """
        super().__init__()

        # Acts as Kaiming weight initalization
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Reduce the input size by 2 to match output size
        self.skip1 = tf.keras.layers.Conv2D(out_channels, 2, strides=2, use_bias=False, kernel_initializer=initializer)

        # Part 1 of the convolution which reduces the spacial area
        self.conv1 = tf.keras.layers.Conv2D(out_channels, 3, strides=2, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm1 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu1 = tf.keras.layers.Activation('relu')

        # Part 2 of the convolution which extracts features from the reduced input
        self.conv2 = tf.keras.layers.Conv2D(out_channels, 3, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm2 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu2 = tf.keras.layers.Activation('relu')

        # Add layer will add together the input and the output
        self.add = tf.keras.layers.Add()

    def call(self, x, training=False):
        """Performs forward pass on the given input.

        Args:
            x (Tensor):      The input of dimensions (N, H, W, C)
            training (bool): Indicates whether the forward pass happens in the training mode

        Returns:
            out (Tensor): Output data of dim (N, H/2, W/2, out_channels)
        """
        x_skip = self.skip1(x)                    # prepare to add the input to the output
        x = self.relu1(self.norm1(self.conv1(x))) # pass input through the first layer
        x = self.norm2(self.conv2(x))             # pass input through the second layer (without ReLU)
        out = self.relu2(self.add([x, x_skip]))   # perform ReLU on the processed input added with the raw input

        return out

In [18]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.model = ResNet()

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.model.call(input_tensor, training)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 100
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 5.723515033721924, Accuracy: 14.0625, Val Loss: 96.11455535888672, Val Accuracy: 8.90000057220459
Iteration 100, Epoch 1, Loss: 1.9524948596954346, Accuracy: 33.555076599121094, Val Loss: 3.626565933227539, Val Accuracy: 17.700000762939453
Iteration 200, Epoch 1, Loss: 1.72105872631073, Accuracy: 39.47450256347656, Val Loss: 2.184948444366455, Val Accuracy: 38.0
Iteration 300, Epoch 1, Loss: 1.5955733060836792, Accuracy: 43.293190002441406, Val Loss: 1.7420998811721802, Val Accuracy: 47.20000076293945
Iteration 400, Epoch 1, Loss: 1.5009397268295288, Accuracy: 46.47755432128906, Val Loss: 1.2915042638778687, Val Accuracy: 54.29999923706055
Iteration 500, Epoch 1, Loss: 1.432699203491211, Accuracy: 48.7618522644043, Val Loss: 1.270981788635254, Val Accuracy: 53.89999771118164
Iteration 600, Epoch 1, Loss: 1.3808494806289673, Accuracy: 50.668155670166016, Val Loss: 1.2471815347671509, Val Accuracy: 56.0
Iteration 700, Epoch 1, Loss: 1.3317935466766357, Accurac

Опишите все эксперименты, результаты. Сделайте выводы.

In [ ]:
  Была реализована следующая архитектура:
        1. `CONV->NORM->RELU->CONV->NORM->RELU` для предварительной обработки входного сигнала для цепочки блоков слоев
        2. `IDENTITY->[BOTTLENECK->IDENTITY] x N`, где каждый блок состоит из произвольного
           где каждый блок состоит из произвольного количества слоев, использующих "skip" соединения
        3. `POOL->DENSE->SOFTMAX`, где выполняется глобальное усреднение пула
           перед вычислением raw scores
В результате произведенной работы были получены необходимые результаты уже на 3 эпохе